In [79]:
# Import packages
import pandas as pd
import numpy as np
import os
import pdfplumber
import re

In [3]:
# Define a function to read in text from pdfs

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [122]:
# Create a function that returns matched patterns
def extract_text(text, pattern):
    match = re.findall(pattern, text)
    #return match[0] if match else np.nan
    return match if len(match)>1 else match[0] if len(match) else np.nan

In [113]:
# Set the patterns
number_pattern = r"Invoice Number:\s*(\d*)"
    
date_pattern = r"Invoice Date:\s*(\b\d{2}[/\-.]\d{2}[/\-.]\d{4}\b)"
    
deemed_amount_pattern = r"Deemed.* £?(\d{1,4}\.\d{2})"

generation_amount_pattern = r"Generation.* £?(\d{1,4}\.\d{2})"

total_amount_pattern = r"Total.* £?(\d{1,4}\.\d{2})"

patterns = [number_pattern, date_pattern, deemed_amount_pattern,
            generation_amount_pattern, total_amount_pattern]

In [5]:
# Change the wd to read in the data files
os.chdir('./Data')

In [6]:
# Save the file names to a variable
file_names = os.listdir()

In [126]:
# Import all of the files and add the date

# Create a df to save data
solar = pd.DataFrame(columns=['file', 'invoice_number', 'invoice_date', 
                              'deemed_amount', 'generation_amount', 'total_amount'])

# Iterate through the files
for file in file_names:
    # Open the files using the pdf function
    text = extract_text_from_pdf(file)
    # Extract the data using the text function
    #number, date, deemed, generation, total = extract_data(text)
    new_row = [extract_text(text, i) for i in patterns]
    
    #new_row = [file, number, date, deemed, generation, total]
    solar.loc[len(solar)] = [file] + new_row
    

# Set the date field to a date field
solar['invoice_date'] = pd.to_datetime(solar['invoice_date'], format='%d.%m.%Y')
# Sort by invoice date
solar = solar.sort_values(by = 'invoice_date')
# Reset the index
solar = solar.reset_index()

In [128]:
# Display the head of the dataframe
solar.head()

,index,file,invoice_number,invoice_date,deemed_amount,generation_amount,total_amount
0,36,FIT_803B2C15112010_181110_113933.pdf,1900005075,2010-11-16,9.08,249.86,258.94
1,29,FIT_803B2C24022011_020311_110050.pdf,1900001602,2011-02-24,4.76,130.92,135.68
2,6,FIT_803B2C10052011_130511_081522.pdf,1900004998,2011-05-12,1.92,52.86,54.78
3,28,FIT_803B2C11082011_120811_094035.pdf,1900009345,2011-08-11,10.52,292.51,303.03
4,45,FIT_803B2C21112011_211111_145211.pdf,1900015792,2011-11-21,11.70,326.92,338.62
